# Finetuning de um Modelo de Linguagem com LoRA

Este notebook descreve o processo de finetuning de um modelo de linguagem utilizando a técnica LoRA (Low-Rank Adaptation). A técnica LoRA é uma abordagem que permite adaptar grandes modelos de linguagem sem a necessidade de treinar todos os parâmetros do modelo, tornando o processo mais eficiente em termos de tempo e recursos computacionais. Vamos seguir passo a passo desde a instalação das bibliotecas necessárias até o treinamento e inferência do modelo.

## Instalação de Bibliotecas

Primeiro, precisamos instalar as bibliotecas essenciais para o nosso projeto. Utilizaremos `bitsandbytes` para aceleração em GPU, `datasets` para carregar conjuntos de dados pré-existentes, `accelerate` para facilitar o uso de múltiplas GPUs, e `loralib` para implementar a técnica LoRA. Adicionalmente, instalamos `transformers` (a principal biblioteca para trabalhar com modelos de linguagem da Hugging Face), `torch`, e `torchvision`.

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
!pip install transformers torch torchvision

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Autenticação no Hugging Face Hub

A Hugging Face oferece uma plataforma para hospedar e compartilhar modelos de aprendizado de máquina. Precisamos autenticar nossa conta para carregar ou salvar modelos. A função notebook_login facilita esse processo de login diretamente do notebook. Para qualquer eventual dúvida com esta parte, [acesse](https://huggingface.co/docs/hub/security-tokens).


In [1]:
from huggingface_hub import notebook_login

notebook_login()

## Verificação de GPUs Disponíveis

Em seguida, verificamos as GPUs disponíveis no sistema com o comando `nvidia-smi -L`. Isso nos permite confirmar que temos os recursos de hardware necessários para treinar nosso modelo de maneira eficiente.

In [3]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1650 (UUID: GPU-eec28b89-0dd4-f4e7-4720-caa347da1457)


## Configuração do Ambiente e Carregamento do Modelo

Agora configuramos o ambiente de execução para usar uma GPU específica (neste caso, a GPU 0) e carregamos o modelo `bloom-560m`, que é um modelo de linguagem treinado pela BigScience. Utilizamos a função `AutoModelForCausalLM.from_pretrained` para carregar o modelo pré-treinado, e `AutoTokenizer.from_pretrained` para carregar o tokenizador correspondente.



In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-560m",
    device_map='auto',  
    force_download=True
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

c:\Users\gabya\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gabya\.cache\huggingface\hub\models--bigscience--bloom-560m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/repos/76/61/766109d3d0b4ee837dfedfab0527ead28aab8af261c2eacbb3c7f7e5d3676920/a8702498162c95d68d2724e7f333c83d7be08de81cfc091455c38730682116d3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1723578073&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMzU3ODA3M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83Ni82MS83NjYxMDlkM2QwYjRlZTgzN2RmZWRmYWIwNTI3ZWFkMjhhYWI4YWYyNjFjMmVhY2JiM2M3ZjdlNWQzNjc2OTIwL2E4NzAyNDk4MTYyYzk1ZDY4ZDI3MjRlN2YzMzNjODNkN2JlMDhkZTgxY2ZjMDkxNDU1YzM4NzMwNjgyMTE2ZDM%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=mV1TvGv4CabQlnm25NxYDgxQJIvcFxsDB5HCUspNSJxfZaE%7E4HL4TiWRLkU-B8aBM3abgNZiqsX4jn48t7qCwfDBwHc0ppzXNDoCDFPiwzuEAoBA0GiMw2tzQdZTIdTFcWp3tylcGusgkMY%7Ex0vKVR4bBdi1fua0KPOAJXIYao3JNOll0pt6SQREmBOk4aTAOIH4zzrYahjJnH1SBxC7pcrdSntQDaffAdN121agr1fC3c27%7

model.safetensors:  32%|###1      | 357M/1.12G [00:00<?, ?B/s]

In [4]:
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

## Congelamento de Parâmetros do Modelo

Para evitar o treinamento de todo o modelo, o que seria computacionalmente caro, congelamos a maioria dos parâmetros utilizando `param.requires_grad = False`. Dessa forma, apenas algumas camadas específicas serão treinadas. Também convertemos alguns parâmetros menores para o formato `float32` para melhorar a estabilidade durante o treinamento. Além disso, habilitamos o checkpointing de gradiente para reduzir o uso de memória durante o treinamento.

In [5]:
for param in model.parameters():
  param.requires_grad = False 
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

In [6]:
model.gradient_checkpointing_enable()  
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  '''
  This class's forward method takes in an input and converts it to float32 form.
  '''
  def forward(self, x):
    return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

## Contagem de Parâmetros Treináveis

Para entender melhor a quantidade de parâmetros que serão ajustados durante o treinamento, criamos uma função que imprime o número total de parâmetros treináveis e a porcentagem desses em relação ao total de parâmetros do modelo.

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

## Configuração e Aplicação de LoRA

O próximo passo é configurar e aplicar a técnica LoRA ao nosso modelo. LoRA permite que ajustemos apenas algumas partes do modelo, como cabeças de atenção, para melhorar a eficiência do treinamento. Definimos uma configuração utilizando `LoraConfig`, especificando os módulos alvo que queremos ajustar, a quantidade de dropout a ser aplicada, e outros parâmetros específicos da LoRA. Em seguida, aplicamos essa configuração ao nosso modelo com `get_peft_model`.

In [8]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 560787456 || trainable%: 0.2804741766549072


## Carregamento e Preparação dos Dados

Para treinar nosso modelo, precisamos de um conjunto de dados. Neste exemplo, utilizamos um dataset de citações em inglês, que carregamos utilizando a função `load_dataset` da biblioteca `datasets`. Modificamos o dataset para adicionar uma coluna de "predição", que é uma combinação da citação com suas tags, e, em seguida, tokenizamos o texto para preparar os dados para o treinamento.

In [9]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")


In [10]:
print(data)
print(data["train"]) 

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})
Dataset({
    features: ['quote', 'author', 'tags'],
    num_rows: 2508
})


In [11]:
def merge_columns(example):
    example["prediction"] = example["quote"] + " ->: " + str(example["tags"])
    return example

train_dataset = data['train']
train_dataset = train_dataset.map(merge_columns)
train_dataset["prediction"][:5]

["“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']",
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.” ->: ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst']",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.” ->: ['human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe']",
 "“So many books, so little time.” ->: ['books', 'humor']",
 "“A room without books is like a body without a soul.” ->: ['books', 'simile', 'soul']"]

In [12]:
train_dataset[0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator'],
 'prediction': "“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']"}

In [13]:
train_dataset = train_dataset.map(lambda samples: tokenizer(samples['prediction']), batched=True)

In [14]:
train_dataset

Dataset({
    features: ['quote', 'author', 'tags', 'prediction', 'input_ids', 'attention_mask'],
    num_rows: 2508
})

## Configuração e Treinamento do Modelo

Agora estamos prontos para configurar o treinamento do modelo. Utilizamos a classe `Trainer` da Hugging Face, que facilita o processo de treinamento. Definimos parâmetros como o tamanho do batch, o número de passos de acumulação de gradiente, o número de passos de aquecimento, e a taxa de aprendizado. Para minimizar o uso de recursos, configuramos o treinamento para rodar apenas por um pequeno número de passos.

In [15]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1, 
        gradient_accumulation_steps=10, 
        warmup_steps=1, 
        max_steps=1, 
        learning_rate=2e-4,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False 

try:
  trainer.train()
except KeyboardInterrupt:
  print("Key board interrupt!")

max_steps is given, it will override any value given in num_train_epochs


  0%|          | 0/1 [00:00<?, ?it/s]

{'loss': 3.9635, 'grad_norm': 1.5149720907211304, 'learning_rate': 0.0, 'epoch': 0.0}
{'train_runtime': 88.2632, 'train_samples_per_second': 0.113, 'train_steps_per_second': 0.011, 'train_loss': 3.9634811878204346, 'epoch': 0.0}


## Carregamento do Modelo Finetunado

Finalmente, para realizar inferências com o modelo finetunado, carregamos o modelo LoRA do Hub utilizando `from_pretrained`, passando tanto o modelo subjacente quanto o tokenizador. Em seguida, criamos um lote de entrada e geramos uma sequência de texto com o modelo.

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "tayyibsupercool/bloom-560m-lora"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model, peft_model_id)

In [14]:
model = model.to('cuda')
batch = tokenizer("I like strawberrys", return_tensors='pt').to('cuda')
with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))




 I like strawberrys, but I don't like them in the garden. I don't like the smell of them in the garden. I don't like the smell of them in the garden. I don't like the smell of them in the garden. I don't like the smell of them
